In [2]:
# getting access to web and scrape the html using def

import urllib
from urllib.request import Request, urlopen

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup

"""[Get the list of symbollist (stock ticker symbols) from IBDlog]
Returns:
  [np array]
"""
def get_company_list(url="aa", attrID="style", attrStr="border-collapse: collapse;"):
    df = ""
    print("Get company symbols from {}".format(url))
#     req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0"})
#     webpage = urllib.request.urlopen(req).read()  # open web pages
#     html = soup(webpage, "lxml")  # html = soup(webpage, "html.parser")
    

    req = Request(url, headers={"User-Agent": "Mozilla/5.0"})
    webpage = urlopen(req).read()
    html = soup(webpage,'lxml')

#     print(html)

    ibd_table = html.find("table", attrs={attrID: attrStr})  # table data のみにアクセス。
    ibd_table_data = ibd_table.find_all("tr")  # access to tr/
    # ibd_tablelh_data

    getheader = []
    for tr in ibd_table_data:
        td = tr.find_all("td")
        row = [tr.text for tr in td]
        getheader.append(row)
        # print(getheader)
        df = pd.DataFrame(getheader)
        df.columns = df.iloc[0]  # Column Name 指定
    df

    return df

'''
# headers={'User-Agent': 'Mozilla/5.0'}
# response = requests.get('https://www.investors.com/data-tables/top-ranked-low-priced-stocks-Sep-13-2021/', headers=headers)
# print(response.status_code)
'''

"\n# headers={'User-Agent': 'Mozilla/5.0'}\n# response = requests.get('https://www.investors.com/data-tables/top-ranked-low-priced-stocks-Sep-13-2021/', headers=headers)\n# print(response.status_code)\n"

In [4]:
# 以下のwebからIBDランキングのデータを取得する。
# 日付はいつも最新ではないので、注意して日付を更新する。
# ここを走らせれば、symbolの抜粋は可能
df_low = get_company_list(
    "https://www.investors.com/data-tables/top-ranked-low-priced-stocks-feb-24-2022/",attrID="class",attrStr="table",)

# print(df_low)
# using dictionary to convert specific columns
convert_dict = {"Eps Rtg": int,"Rel Str": float,"Vol% Chg": int,"Cmp Rtg": int, "Price Chg": float, "Acc Dis": str,}

df_low = df_low[df_low["Name"] != "Name"]
df_low = df_low.astype(convert_dict)
df_low["Total"] = df_low["Cmp Rtg"] + df_low["Eps Rtg"] + df_low["Rel Str"]
df_low = df_low[    (df_low["Eps Rtg"] > 60) & (df_low["Rel Str"] > 60) & (df_low["Cmp Rtg"] > 70)]
# df_low=df_low.sort_values('Vol% Chg',ascending=False).head(20)
# print(df_low.head(10))
# df_low.sort_values('Price Chg',ascending=False).head(20)
df_low = df_low.sort_values(["Vol% Chg", "Price Chg", "Price"], ascending=[0, 1, 1])

# df_low.loc[df_low['Eps Rtg']>90]
df_low

Get company symbols from https://www.investors.com/data-tables/top-ranked-low-priced-stocks-feb-24-2022/


,Cmp Rtg,Eps Rtg,Rel Str,Acc Dis,Yr Hi,Name,Symbol,Price,Price Chg,Vol% Chg,Total
440,80,71,71.0,B-,10.4,ProvidentAqA,PAQC,9.77,0.02,853,222.0
555,82,64,84.0,B,7.9,US Energy,USEG,4.13,0.02,532,230.0
98,94,81,95.0,B+,24.0,CalifBancorp,CALB,22.41,-0.03,356,270.0
361,88,87,84.0,C,27.8,MidlfieldBnc,MBCN,25.40,-0.22,333,259.0
477,85,71,65.0,B+,28.6,SagaCom,SGA,23.00,0.35,319,221.0
...,...,...,...,...,...,...,...,...,...,...,...
392,91,91,92.0,B,26.3,OldPoint,OPOF,24.75,0.00,-93,274.0
399,98,99,92.0,A,42.6,OrangeCtyBnc,OBT,40.34,-0.26,-95,289.0
488,75,62,67.0,C-,11.3,ScvxCorpA,SCVX,9.91,-0.01,-95,204.0
302,77,71,76.0,D+,9.9,ItiquiraAcqA,ITQ,9.74,0.00,-97,224.0


In [ ]:
# IBD Large/Mid-Cap Leaders Index

url = "https://www.investors.com/data-tables/ibd-large-mid-cap-leaders-index-apr-29-2021/"
url = "https://www.investors.com/data-tables/timesaver-table-apr-29-2021/"
url = "https://www.investors.com/data-tables/dividend-leaders-apr-29-2021/"
url ="https://www.investors.com/ibd-indexes/ibd-breakout-stocks-index/"

df_largemid = get_company_list(url,attrID="class", attrStr="table",)
df_low = df_largemid
df_low = df_low.drop(df_low.index[[0]])
df_low = df_low.fillna(value=np.nan)
for i in df_low['Name']:
    print(i)

In [ ]:
df_low.describe()

In [ ]:
# df_low = pd.read_csv('/Users/yutaka/Downloads/america_2021-05-01.csv').values.tolist()
# df_low = pd.read_csv('/Users/yutaka/Downloads/america_2021-05-01.csv')
# df_low = df_low.rename({'Ticker': 'Symbol'}, axis=1)  # new method
# df_low

In [5]:
"""
finviz　data download into dataframe.
"""

import datetime
import os
import sys
from urllib.request import Request, urlopen

import matplotlib.pyplot as plt
import mplfinance as mpf
import yfinance as yf
from bs4 import BeautifulSoup as soup

In [8]:
end_date = datetime.datetime.now()
duration = 1000
start_date = end_date - datetime.timedelta(days=duration)

###########folder edit#############
dir = os.path.join("/Users/yutaka/Google ドライブ/New Folder/Python_Output",    end_date.strftime("%y%m%d") + "_low-priced",)
if not os.path.exists(dir):
    os.makedirs(dir)

# symbol=df_list[0].unique()
df = pd.DataFrame()
df1 = pd.DataFrame()

for ind in df_low.index[:]:

    i = df_low["Symbol"][ind]  # symbolのindex＃を巡回
    # for i in list(ETF_list[:,0]):      ##nparrayで変換しないと動かない。 index の巡回でもいけるはず。

    pd.set_option("display.max_colwidth", 25)
    pd.set_option("display.max_rows", 75)

    url = "http://finviz.com/quote.ashx?t=" + i.lower()
    print(url)
    req = Request(url, headers={"User-Agent": "Mozilla/5.0"})
    try:
        webpage = urlopen(req).read()
        html = soup(webpage, "html.parser")

        def get_fundamentals():
            try:
                # Find fundamentals table
                fundamentals = pd.read_html(str(html), attrs={"class": "snapshot-table2"})[0]

                # Clean up fundamentals dataframe
                fundamentals.columns = ["0","1","2","3","4","5","6", "7","8","9","10","11", ]
                colOne = []
                colLength = len(fundamentals)
                for k in np.arange(0, colLength, 2):
                    colOne.append(fundamentals[f"{k}"])
                attrs = pd.concat(colOne, ignore_index=True)

                colTwo = []
                colLength = len(fundamentals)
                for k in np.arange(1, colLength, 2):
                    colTwo.append(fundamentals[f"{k}"])
                vals = pd.concat(colTwo, ignore_index=True)

                fundamentals = pd.DataFrame()
                fundamentals["Attributes"] = attrs
                fundamentals["Values"] = vals
                fundamentals = fundamentals.set_index("Attributes")
                return fundamentals

            except Exception as e:
                return e

        df1 = get_fundamentals()
    #         print(df1)

    except Exception as e:
        print(e.args)

    try:
        tb10 = yf.download(i, interval="1wk", start=start_date, end=end_date)  # 1wk
        dfg = pd.DataFrame(tb10)
        dfg = dfg.drop(columns="Adj Close", axis=1)
        dfg.columns = ["Open", "High", "Low", "Close", "Volume"]

        fig, axlist = mpf.plot(
            dfg[1:duration],
            type="candle",
            volume=True,
            title=i,
            mav=(10, 20, 50, 100, 200),
            figratio=(18, 10),
            style="yahoo",
            savefig=dict(
                fname=os.path.join(dir, end_date.strftime("%y%m%d_") + str(i) + ".png"),
                dpi=200,
                pad_inches=0.1,
            ),
            returnfig=True,
        )

    except Exception as e:
        print("Could not download the finance data from Yahoofinance;{}".format(e.args))
        continue

    # 全てのdataを
    df1 = df1.rename(columns={"Values": str(i)})
    df = pd.concat([df1, df], axis=1)  # axix=1 means adding columns
    # 重複カラム削除
    df = df.loc[:, ~df.columns.duplicated()]
    df.to_csv(os.path.join(dir, end_date.strftime("%y%m%d_merge") + ".csv"))
    df.T.to_csv(os.path.join(dir, end_date.strftime("%y%m%d_merge") + "T.csv"))

print("Done!")

http://finviz.com/quote.ashx?t=paqc
[*********************100%***********************]  1 of 1 completed
http://finviz.com/quote.ashx?t=useg
[*********************100%***********************]  1 of 1 completed
http://finviz.com/quote.ashx?t=calb
()
[*********************100%***********************]  1 of 1 completed
http://finviz.com/quote.ashx?t=mbcn
()
[*********************100%***********************]  1 of 1 completed
http://finviz.com/quote.ashx?t=sga
[*********************100%***********************]  1 of 1 completed
http://finviz.com/quote.ashx?t=out
[*********************100%***********************]  1 of 1 completed
http://finviz.com/quote.ashx?t=vist
[*********************100%***********************]  1 of 1 completed
http://finviz.com/quote.ashx?t=invh
[*********************100%***********************]  1 of 1 completed
http://finviz.com/quote.ashx?t=hbm
[*********************100%***********************]  1 of 1 completed
http://finviz.com/quote.ashx?t=gcbc
[***************

In [2]:
df_funda = df.T
df_funda
df_funda.head(10)

NameError: name 'df' is not defined

In [ ]:
#重複カラム削除
df.loc[:,~df.columns.duplicated()]

In [1]:
convert_header = {"Income": str, "EPS Q/Q": str}
df_funda = df_funda.astype(convert_header)
df_funda.dtypes

df_funda["EPS Q/Q"] = df_funda["EPS Q/Q"].str.replace("%", "")
df_funda=df_funda[df_funda["EPS Q/Q"] != "-"]

convert_header = {"Income": str, "EPS Q/Q": float}
df_funda = df_funda.astype(convert_header)
df_funda.dtypes

# df["EPS Q/Q"]= df["EPS Q/Q"].astype(float)
df_funda[["Income",'Sales','Sales Q/Q','EPS Q/Q','Inst Own','Inst Trans','Rel Volume','RSI (14)']].sort_values(["EPS Q/Q"], ascending=[0]).head(50)
# df_funda.dtypes

NameError: name 'df_funda' is not defined

In [ ]:

import datetime
import os
import sys
from urllib.request import Request, urlopen

import matplotlib.pyplot as plt
import mplfinance as mpf
import yfinance as yf
from bs4 import BeautifulSoup as soup


url = "http://finviz.com/quote.ashx?t=RIOT"  #+ i.lower()
print(url)
req = Request(url, headers={"User-Agent": "Mozilla/5.0"})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")
print(html)

# def get_fundamentals():
#     try:
#         # Find fundamentals table
#         fundamentals = pd.read_html(str(html), attrs={"class": "snapshot-table2"})[0]

#         # Clean up fundamentals dataframe
#         fundamentals.columns = ["0","1","2","3","4","5","6","7","8","9","10","11",]
#         colOne = []
#         colLength = len(fundamentals)
#         for k in np.arange(0, colLength, 2):
#             colOne.append(fundamentals[f"{k}"])
#         attrs = pd.concat(colOne, ignore_index=True)

#         colTwo = []
#         colLength = len(fundamentals)
#         for k in np.arange(1, colLength, 2):
#             colTwo.append(fundamentals[f"{k}"])
#         vals = pd.concat(colTwo, ignore_index=True)

#         fundamentals = pd.DataFrame()
#         fundamentals["Attributes"] = attrs
#         fundamentals["Values"] = vals
#         fundamentals = fundamentals.set_index("Attributes")
#         return fundamentals

#     except Exception as e:
#         return e

# df1 = get_fundamentals()
# #         print(df1)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.keys import Keys # webdriverからスクレイピングで使用するキーを使えるようにする。
from webdriver_manager.chrome import ChromeDriverManager
import time
import pyautogui

options = webdriver.ChromeOptions()
options.add_argument("--headless")
DRIVER_PATH = "/Users/yutaka/Downloads/chromedriver"

def CreatFolder():
    dir = os.path.join("/Users/yutaka/Google ドライブ/New Folder/Python_Output",
                       end_date.strftime("%y%m%d") + "_Skew",)
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# webdriver = webdriver.Chrome()
def WebCapture(symbol):
    
    symbol= symbol.replace("/","")
    
    driver.find_element_by_xpath("//*[@id='siteheader-outer-second']/div/input").send_keys(symbol)
    driver.find_element_by_css_selector("input.sym_lookup_btn").click()
    time.sleep(0.3)

    screenshot=pyautogui.screenshot(region=(50,50,1500,975))
    screenshot.save(CreatFolder() + '/' + symbol+'.png')
    driver.find_element_by_xpath("//*[@id='siteheader-outer-second']/div/input").clear()

driver = webdriver.Chrome(executable_path=DRIVER_PATH) # , options=options)
URL = f'https://marketchameleon.com/Overview/AAPL/VolatilitySkew/'
driver.get(URL)

# for symb in df_funda.index.values[:]:
import pandas as pd
Symbol_list = pd.read_csv('/Users/yutaka/Downloads/america_2021-04-24.csv').values.tolist()
for ticker in Symbol_list[:]:
    WebCapture(ticker[0])

# for symb in df_low['Symbol']:
#     WebCapture(symb)
driver.quit()


In [ ]:
for ticker in Symbol_list[:5]:
    print(type(ticker[0]))
#     WebCapture(ticker[0])

In [ ]:
for symb in df_low['Symbol']:
    print(type(symb))

In [ ]:
import json
import datetime
import requests 
from bs4 import BeautifulSoup


url = 'https://au.investing.com/common/modules/js_instrument_chart/api/data.php?pair_id=1&pair_id_for_news=1&chart_type=area&pair_interval=300&candle_count=120&events=yes&volume_series=yes&period='
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0',
    'X-Requested-With': 'XMLHttpRequest',
    'Referer': 'https://au.investing.com/currencies/eur-usd'}
data = requests.get(url, headers=headers).json()

In [ ]:
data